In [2]:
%reload_ext autoreload
%autoreload 2

import os, sys
import pickle

import numpy as np
from scipy import interpolate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../../matplotlibrc')

In [3]:
if os.environ['USER'] == 'yitians' and 'submit' in os.uname().nodename:
    os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/DM21cm/data'
sys.path.append(os.environ['DH_DIR'])
sys.path.append('..')

In [43]:
import main

import dm21cm.physics as phys
import darkhistory.physics as dhphys

## 1. DarkHistory runs

In [18]:
base_21cmfast = np.load(f'../../data/run_info/base_records.npy', allow_pickle=True).item()

start_rs = base_21cmfast['z'][0] + 1
end_rs = base_21cmfast['z'][-1] + 1

### 1.1 Get recfast values

In [19]:
run_name = 'base_recfast'

rs_s = np.geomspace(start_rs, end_rs, 100)
x_e_s = dhphys.xHII_std(rs_s)
T_k_s = dhphys.Tm_std(rs_s) / phys.kB

In [20]:
arr_records = {
    'z' : rs_s - 1,
    'x_e' : x_e_s,
    'T_k' : T_k_s,
}
print(run_name)

base_recfast


In [22]:
np.save(f'../../data/run_info/DH/{run_name}_records', arr_records)

### 1.2 force case A with fake reionization

In [42]:
run_name = 'base_caseA_in_reion'

soln = main.evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-50, primary='phot_delta',
    start_rs=50, end_rs=48,
    coarsen_factor=12, backreaction=False, reion_switch=True,
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    reion_rs=49,
    photoion_rate_func = [lambda rs: 0.] * 3,
    photoheat_rate_func = [lambda rs: 0.] * 3,
    use_tqdm=False, verbose=1
)

Loading time: 0.016 s
Initialization time: 0.047 s
B1.1743e-02 B4.2103e-03 B1.1743e-02 B1.1743e-02 B1.1632e-02 B4.1372e-03 B1.1632e-02 B1.1632e-02 B1.1632e-02 B4.1364e-03 B1.1632e-02 B1.1632e-02 B1.1521e-02 B4.0639e-03 B1.1521e-02 B1.1521e-02 B1.1521e-02 B4.0631e-03 B1.1521e-02 B1.1521e-02 B1.1603e-02 B4.1171e-03 B1.1603e-02 B1.1603e-02 B1.1508e-02 B4.0553e-03 B1.1508e-02 B1.1508e-02 B1.1508e-02 B4.0547e-03 B1.1508e-02 B1.1508e-02 A4.0547e-03 A1.0137e-03 B1.1508e-02 A4.0547e-03 A1.0137e-03 A1.0137e-03 A4.0264e-03 A1.0066e-03 B1.1465e-02 A4.0264e-03 A1.0066e-03 A1.0066e-03 A4.0263e-03 A1.0066e-03 B1.1465e-02 A4.0263e-03 A1.0066e-03 A1.0066e-03 A4.0263e-03 A1.0066e-03 B1.1465e-02 A4.0263e-03 A1.0066e-03 A1.0066e-03 A3.9601e-03 A9.9003e-04 B1.1362e-02 A3.9601e-03 A9.9003e-04 A9.9003e-04 A3.9595e-03 A9.8987e-04 B1.1362e-02 A3.9595e-03 A9.8987e-04 A9.8987e-04 A3.9908e-03 A9.9771e-04 B1.1410e-02 A3.9908e-03 A9.9771e-04 A9.9771e-04 A3.9906e-03 A9.9766e-04 B1.1410e-02 A3.9906e-03 A9.9766e-04 A

In [31]:
pickle.dump(soln, open(f'../../data/run_info/DH/{run_name}_soln.p', 'wb'))

### 1.3 custom injection

In [ ]:
import darkhistory.physics as dhphys
from darkhistory.spec import pppc
from config import load_data

In [ ]:
# custom xe levels
def x_reion_func(rs):
    # extrapolate z low so that darkhistory doesn't complain
    z_arr = base_records['z'].copy()
    z_arr = np.append(z_arr, 2 * z_arr[-1] - z_arr[-2])
    x_e_arr = base_records['x_e'].copy()
    x_e_arr = np.append(x_e_arr, x_e_arr[-1])
    return interpolate.interp1d(1 + z_arr, x_e_arr)(rs)

In [21]:
run_name = 'phph_turnon_z35_comp'

mDM = 1e10
sigmav = 1e-23
struct_boost = phys.struct_boost_func('erfc 1e-3')

photeng = load_data('binning')['phot']
eleceng = load_data('binning')['elec']

def in_spec_elec_func(rs):
    in_spec_elec = pppc.get_pppc_spec(1e10, eleceng, 'phot_delta', 'elec')
    in_spec_elec.rs = rs
    if rs > 36:
        in_spec_elec *= 0
    return in_spec_elec

def in_spec_phot_func(rs):
    in_spec_phot = pppc.get_pppc_spec(1e10, photeng, 'phot_delta', 'phot')
    in_spec_phot.rs = rs
    if rs > 36:
        in_spec_phot *= 0
    return in_spec_phot

def rate_func_N(rs):
    return (
        dhphys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)
        * struct_boost(rs) / (2*mDM)
    )
def rate_func_eng(rs):
    return (
        dhphys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav) 
        * struct_boost(rs)
    )

In [25]:
soln = main.evolve(
    in_spec_elec = in_spec_elec_func,
    in_spec_phot = in_spec_phot_func,
    rate_func_N = rate_func_N,
    rate_func_eng = rate_func_eng,
    start_rs=3000, end_rs=7,
    coarsen_factor=12, backreaction=True, reion_switch=True,
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    verbose=1,
    use_tqdm=False
)

Loading time: 0.000 s
Initialization time: 0.004 s
Main loop time: 49.761 s


## 2. Make arr_records and save

In [25]:
arr_records = {
    'z' : soln['rs'] - 1,
    'x_e' : soln['x'][:,0],
    'T_k' : soln['Tm']/phys.kB,
    'f_heat' : soln['f']['low']['heat'] + soln['f']['high']['heat'],
    'f_ion' : soln['f']['low']['H ion']  + soln['f']['high']['H ion'] + \
              soln['f']['low']['He ion'] + soln['f']['high']['He ion'],
    'f_exc' : soln['f']['low']['exc'] + soln['f']['high']['exc'],
}

z_low, z_high = 6, 44
z_arr = arr_records['z'][::-1]
i_low = np.searchsorted(z_arr, z_low)
i_high = np.searchsorted(z_arr, z_high)

for k in arr_records.keys():
    arr_records[k] = arr_records[k][::-1][i_low:i_high]
    
print(run_name)

base_caseA_in_reion


In [27]:
np.save(f'../../data/run_info/DH/{run_name}_records', arr_records)